<a href="https://colab.research.google.com/github/hmg8906/CienciaDatosCN/blob/main/RUSLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project="hmgarcia56ee1")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [ ]:
from ee.batch import Export
from datetime import datetime
import time

def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(5)


In [ ]:
def rusle(years, roi, prefix, folder, scale):
    """
    Estima la Erosión mediante RUSLE para diferentes años y exporta los resultados a Google Drive.

    Args:
    - years: Lista de años (e.g., [2004, 2014, 2024]).
    - roi (ee.Geometry): Región de interés.
    - prefix: Prefijo para nombrar las imágenes exportadas.
    - folder: Carpeta en Google Drive donde se guardarán los resultados.
    - scale: Escala espacial (en metros).
    """
    for year in years:
        print(f"Calculando RUSLE para el año {year}...")

               # -----------
        # R factor (usando precipitación CHIRPS para el año específico y ecuación Bols(1974))
        # -----------

        # Filtrar la colección de imágenes de precipitación diaria para un año específico (year)
        precip_daily = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")\
                        .filterBounds(roi)\
                        .filterDate(f'{year}-01-01', f'{year}-12-31')\


        # Calcular la precipitación anual
        precip_annual = precip_daily.sum().clip(roi)

        # factor R
        factorR = precip_annual.pow(2).multiply(2.5).divide(precip_annual.multiply(7.3).add(73)).rename('factorR')

        # -----------
        # K factor
        # -----------
        sand = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(roi)
        silt = ee.Image('users/aschwantes/SLTPPT_I').divide(100).clip(roi)
        clay = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(roi)
        morg = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').multiply(0.58).clip(roi)
        sn1 = sand.expression('1 - b0 / 100', {'b0': sand})
        orgcar = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').clip(roi)

        soil = ee.Image([sand, silt, clay, morg, sn1, orgcar]).rename(['sand', 'silt', 'clay', 'morg', 'sn1', 'orgcar'])
        factorK = soil.expression(
            '(0.2 + 0.3 * exp(-0.0256 * SAND * (1 - (SILT / 100)))) * (1 - (0.25 * CLAY / (CLAY + exp(3.72 - 2.95 * CLAY)))) * (1 - (0.7 * SN1 / (SN1 + exp(-5.51 + 22.9 * SN1))))',
            {
                'SAND': soil.select('sand'),
                'SILT': soil.select('silt'),
                'CLAY': soil.select('clay'),
                'MORG': soil.select('morg'),
                'SN1': soil.select('sn1'),
                'CORG': soil.select('orgcar')
            }
        ).rename('factorK')

        # -----------
        # C factor (calculado a partir de NDVI para cada año)
        # -----------
        S2_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                         .filterBounds(roi)\
                         .filterDate(f'{year}-01-01', f'{year}-12-31')\
                         .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)  # Filtrar imágenes con <10% de nubes

        # Obtener la mediana de la colección de imágenes de Sentinel 2
        S2_median = S2_collection.median().clip(roi)

        # Calcular el NDVI usando las bandas Red (B4) y NIR (B5)
        RED = S2_median.select('B3')
        NIR = S2_median.select('B8')
        SWIR = S2_median.select('B11')

        ndvi = NIR.subtract(RED).divide(NIR.add(RED)).rename('NDVI')
        ndwi = SWIR.subtract(NIR).divide(SWIR.add(NIR)).rename('NDWI')

        # Crear una máscara que excluya valores de NDVI menores a 0 (cuerpos de agua)
        # ndvi_masked = ndvi.updateMask(ndvi.gte(0))

        factorC = ee.Image(1)  # Inicializar el factorC
                # C factor calculado a partir del NDVI
        factorC = ee.Image(1)
        factorC = factorC.expression("1-ndvi/2**(1+ndvi)",
        {
                                'ndvi': ndvi
                                  }).rename('factorC')
        # -----------
        # LS factor
        # -----------
        # Cargar el DEM y el factor de acumulación de flujo (usando SRTM)
        dem = ee.Image("USGS/SRTMGL1_003").clip(roi)

        # Aquí se usa un vecindario de 3 píxeles, puedes ajustarlo según sea necesario
        filled_dem = dem.unmask().focal_mean(8, kernelType='circle', iterations=1)

        # Factor de acumulación enmascarado para evitar cuerpos de agua
        # facc = ee.Image("WWF/HydroSHEDS/15ACC").clip(roi).updateMask(ndvi_masked)
        facc = ee.Image("projects/hmgarcia56ee1/assets/flow_acc").clip(roi)

        # Calcular la pendiente
        slope = ee.Terrain.slope(filled_dem).multiply(0.0174533)

        # Crear imagen con factores
        ls_factors = ee.Image([facc, slope]).rename(['facc', 'slope'])

        # Calcular el factor LS
        factorLS = ls_factors.expression(
            '1.4 * ((FACC / 22.13) ** 0.4) * (sin(SLOPE) / 0.0896) ** 1.3',
            {
                'FACC': ls_factors.select('facc'),
                'SLOPE': ls_factors.select('slope')
            }
        ).rename('factorLS')

        # -----------
        # Cálculo de la erosión (A)
        # -----------

        erosion = factorC.multiply(factorR).multiply(factorLS).multiply(factorK).rename('A')
        erosion_output = ee.Image([erosion,factorC,factorR,factorLS,factorK,ndvi,ndwi]).rename(['A','C','R','LS','K','NDVI','NDWI',])

        # -----------
        # Exportar la imagen a Google Drive
        # -----------

        task = ee.batch.Export.image.toDrive(
            image=erosion_output,
            description=f'{prefix}_{year}_erosion',
            folder=folder,
            scale=scale,
            region=roi,
            maxPixels=1e13
        )
        task.start()
        print(f'Tarea de exportación para el año {year} iniciada.')
        ee_monitoring(task)





In [ ]:
roi = ee.FeatureCollection("projects/hmgarcia56ee1/assets/BG2").geometry()
years = [2019,2023]
rusle(years, roi, 'RUSLE', 'RUSLE_BG', scale=30)

Calculando RUSLE para el año 2019...
Tarea de exportación para el año 2019 iniciada.
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR).
Sondeo de la tarea (id: YWHQXPOG6VF43ZPX5AUAHNPR